In [1]:
import os
import pandas as pd

In [2]:
rna_tsne = 'RNAseq_tsne_results.tsv'
rna_nmf = 'RNAseq_nmf_results.tsv'
rna_pca = 'RNAseq_pca_results.tsv'
rna_vae = '../../../vae_pancancer/data/encoded_rnaseq_onehidden_warmup_batchnorm.tsv'

copy_pca = 'CopyNumber_pca_results.tsv'
copy_tsne = 'CopyNumber_tsne_results.tsv'
copy_nmf = 'CopyNumber_nmf_results.tsv'

clinical = "combined_clinical_encoded.tsv"

In [3]:
rna_tsne_df = pd.read_table(rna_tsne, index_col=0)
rna_nmf_df = pd.read_table(rna_nmf, index_col=0)
rna_pca_df = pd.read_table(rna_pca, index_col=0)
rna_vae_df = pd.read_table(rna_vae, index_col=0)

copy_pca_df = pd.read_table(copy_pca, index_col=0)
copy_tsne_df = pd.read_table(copy_tsne, index_col=0)
copy_nmf_df = pd.read_table(copy_nmf, index_col=0)

clinical_df = pd.read_table(clinical, index_col=0)

In [4]:
real_clinical_df = clinical_df.iloc[:, range(101, 118)]
real_clinical_df.index = clinical_df['sample_id']
real_clinical_df.head(2)

,sample_base,platform,portion_id,age_at_diagnosis,stage,vital_status,race,acronym,disease,organ,drug,ethnicity,percent_tumor_nuclei,gender,sample_type,analysis_center,year_of_diagnosis
sample_id,,,,,,,,,,,,,,,,,
TCGA-02-0047-01,TCGA-02-0047,Illumina HiSeq,TCGA-02-0047-01A-01,78.0,NaN,dead,white,GBM,Glioblastoma multiforme,Brain,NaN,not hispanic or latino,90.0,male,Primary Tumor,BI,2005.0
TCGA-02-0055-01,TCGA-02-0055,Illumina HiSeq,TCGA-02-0055-01A-01,62.0,NaN,dead,white,GBM,Glioblastoma multiforme,Brain,Temozolomide,not hispanic or latino,95.0,female,Primary Tumor,BI,2005.0


In [5]:
rna_tsne_df = rna_tsne_df.add_suffix('_rna_tsne')
rna_nmf_df = rna_nmf_df.add_suffix('_rna_nmf')
rna_pca_df = rna_pca_df.add_suffix('_rna_pca')
rna_vae_df = rna_vae_df.add_suffix('_rna_vae')

copy_pca_df = copy_pca_df.add_suffix('_copy_pca')
copy_tsne_df = copy_tsne_df.add_suffix('_copy_tsne')
copy_nmf_df = copy_nmf_df.add_suffix('_copy_nmf')

In [6]:
all_data_combined = (rna_tsne_df.merge(rna_nmf_df, how = 'inner', left_index=True, right_index=True)
                     .merge(rna_pca_df, how = 'inner', left_index=True, right_index=True)
                     .merge(rna_vae_df, how = 'inner', left_index=True, right_index=True)
                     .merge(copy_pca_df, how = 'inner', left_index=True, right_index=True)
                     .merge(copy_tsne_df, how = 'inner', left_index=True, right_index=True)
                     .merge(copy_nmf_df, how = 'inner', left_index=True, right_index=True)
                     .merge(real_clinical_df, how = 'inner', left_index=True, right_index=True))

In [7]:
all_data_combined.to_csv('data_feature_encodings.tsv', sep='\t')